# Bayesian Network Structure Learning  
### Using UCI Breast Cancer Diagnostic Dataset (WDBC)

In this notebook, I implemented **Bayesian Network structure and parameter learning completely from scratch**, without using pgmpy or sklearn BN modules.

Our pipeline includes:

- Data loading and preprocessing  
- Discretization of continuous features  
- Manual Chi-square conditional independence test  
- PC-Skeleton structure discovery  
- Hill-Climbing score-based structure learning  
- Manual Maximum Likelihood CPT estimation  
- Simple BN inference for classification  
- Model evaluation  



In [16]:
import pandas as pd

df = pd.read_csv("data.csv")   # your file name

print(df.shape)
print(df.columns)


(569, 33)
Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')


In [17]:
# drop ID
df = df.drop(columns=["id"])

# Convert diagnosis (M=malignant, B=benign)
df["diagnosis"] = df["diagnosis"].map({"B": "benign", "M": "malignant"})


In [18]:
df.isna().sum()


diagnosis                    0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_se                    0
texture_se                   0
perimeter_se                 0
area_se                      0
smoothness_se                0
compactness_se               0
concavity_se                 0
concave points_se            0
symmetry_se                  0
fractal_dimension_se         0
radius_worst                 0
texture_worst                0
perimeter_worst              0
area_worst                   0
smoothness_worst             0
compactness_worst            0
concavity_worst              0
concave points_worst         0
symmetry_worst               0
fractal_dimension_worst      0
Unnamed: 32                569
dtype: i

In [19]:
df = df.drop(columns=["Unnamed: 32"])


In [20]:
print(df.isna().sum().sum())  # Should print 0


0


In [21]:
print(df.shape)
print(df.columns)

(569, 31)
Index(['diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')


In [22]:
from sklearn.preprocessing import KBinsDiscretizer

feature_cols = df.columns[1:]   # all columns except diagnosis

disc = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')
df[feature_cols] = disc.fit_transform(df[feature_cols]).astype(int).astype(str)


c:\Users\madhu\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(


In [23]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["diagnosis"])


In [37]:
selected = [
    "diagnosis",
    "radius_mean",
    "texture_mean",
    "perimeter_mean",
    "area_mean",
    "smoothness_mean",
    "compactness_mean",
    "concavity_mean",
    "symmetry_mean"
]


In [38]:
train_df = train_df[selected]
test_df = test_df[selected]
vars = selected


### Conditional Independence Test (Chi-Square)
This function checks whether two variables are statistically independent.  
I used chi-square tests because they work well for discrete variables.  
This test is the foundation of the PC algorithm.


In [39]:
from scipy.stats import chi2_contingency
from itertools import combinations

def chi2_test(a, b):
    table = pd.crosstab(a, b)
    if table.shape[0] < 2 or table.shape[1] < 2:
        return 1.0  # not enough data
    chi2, p, _, _ = chi2_contingency(table)
    return p

def ci_test(df, X, Y, cond_set=None, alpha=0.05):
    if not cond_set:
        return chi2_test(df[X], df[Y]) >= alpha
    
    grouped = df.groupby(cond_set)
    p_values = []
    for _, subset in grouped:
        if len(subset) < 5:
            continue
        p = chi2_test(subset[X], subset[Y])
        p_values.append(p)
    
    if len(p_values) == 0:
        return False  # treat dependent
    
    return np.mean(p_values) >= alpha


### PC Algorithm (Skeleton Discovery)
The PC algorithm finds which variables are connected by removing edges if variables are conditionally independent.  
I tested independence with conditioning sets of small size to keep computation fast.  
The output is an undirected graph structure representing likely causal connections.


In [40]:
def pc_skeleton(df, vars, alpha=0.05, max_cond=1):
    graph = {v: set(vars) - {v} for v in vars}
    sep_sets = {}

    for l in range(0, max_cond+1):
        for X in vars:
            for Y in list(graph[X]):
                if X == Y: continue
                neighbors = list(graph[X] - {Y})

                if len(neighbors) < l:
                    continue

                for cond in combinations(neighbors, l):
                    if ci_test(df, X, Y, list(cond), alpha):
                        graph[X].remove(Y)
                        graph[Y].remove(X)
                        sep_sets[(X,Y)] = cond
                        sep_sets[(Y,X)] = cond
                        break
    return graph


In [41]:
import numpy as np

### Cycle Detection for DAG Validity
Bayesian Networks must not contain cycles, so we check if a graph contains any directed loops.  
I performed a DFS-based cycle check after every structural change.  
This ensures the learned network is a valid DAG.


### BIC Score Calculation
The BIC score measures how well a given network structure fits the data while penalizing complexity.  
Higher BIC scores indicate better models.  
This function is used by hill-climbing to decide which edge operations improve the model.


### Hill-Climbing Structure Learning
Hill-climbing searches for the best DAG by trying to add, remove, or reverse edges.  
At each step, we keep the change only if it improves the BIC score.  
This algorithm helps us find a good network structure efficiently.


In [42]:
def has_cycle(edges, vars):
    parents = {v: [] for v in vars}
    for u, v in edges:
        parents[v].append(u)
    
    visited = {v:0 for v in vars}

    def dfs(v):
        if visited[v] == 1:
            return True
        visited[v] = 1
        for p in parents[v]:
            if dfs(p):
                return True
        visited[v] = 2
        return False

    return any(dfs(v) for v in vars)


def bic_score(df, edges, vars):
    N = len(df)
    parents = {v: [] for v in vars}

    for u, v in edges:
        parents[v].append(u)

    LL = 0
    num_params = 0

    for child in vars:
        par = parents[child]
        child_states = sorted(df[child].unique())

        if not par:
            counts = df[child].value_counts()
            total = sum(counts.values)

            for state in child_states:
                c = counts.get(state, 0)
                if c > 0:
                    p = c / total
                    LL += c * np.log(p)

            num_params += (len(child_states) - 1)
            continue

        # with parents
        grouped = df.groupby(par)

        for _, subset in grouped:
            total = len(subset)
            if total == 0:
                continue

            counts = subset[child].value_counts()

            for state in child_states:
                c = counts.get(state, 0)
                if c > 0:
                    p = c / total
                    LL += c * np.log(p)

            num_params += (len(child_states) - 1)

    return LL - 0.5 * num_params * np.log(N)



def hill_climb(df, vars):
    edges = set()
    best = bic_score(df, edges, vars)
    improved = True

    while improved:
        improved = False
        best_move = None

        for X in vars:
            for Y in vars:
                if X == Y: 
                    continue

                # Try add edge
                if (X,Y) not in edges:
                    new_edges = edges | {(X,Y)}
                    if not has_cycle(new_edges, vars):
                        score = bic_score(df, new_edges, vars)
                        if score > best:
                            best = score
                            best_move = ("add", (X,Y))

                # Try remove
                if (X,Y) in edges:
                    new_edges = edges - {(X,Y)}
                    score = bic_score(df, new_edges, vars)
                    if score > best:
                        best = score
                        best_move = ("remove", (X,Y))

        if best_move:
            typ, e = best_move
            if typ == "add":
                edges.add(e)
            else:
                edges.remove(e)
            improved = True

    return edges


### Parameter Learning (MLE for CPTs)
After learning the structure, we calculate the conditional probability tables (CPTs) for each node.  
I computed probabilities directly from frequency counts in the training data.  
These CPTs are required for performing inference.


In [43]:
def learn_cpts(df, edges, vars):
    parents = {v: [] for v in vars}
    for u,v in edges:
        parents[v].append(u)

    cpts = {}

    for child in vars:
        pars = parents[child]

        if not pars:
            counts = df[child].value_counts()
            total = counts.sum()
            probs = {val: counts[val]/total for val in counts.index}
            cpts[child] = {"parents": [], "table": {(): probs}}
            continue

        cpt_table = {}
        grouped = df.groupby(pars)

        for par_vals, subset in grouped:
            par_vals = par_vals if isinstance(par_vals, tuple) else (par_vals,)
            counts = subset[child].value_counts()
            total = counts.sum()
            probs = {v: counts.get(v,0)/total for v in df[child].unique()}
            cpt_table[par_vals] = probs

        cpts[child] = {"parents": pars, "table": cpt_table}

    return cpts


In [44]:
def predict_row(cpts, row, target="Class"):
    info = cpts[target]
    parents = info["parents"]

    if not parents:
        probs = list(info["table"].values())[0]
        return probs

    par_vals = tuple(row[p] for p in parents)
    if par_vals not in info["table"]:
        return {c: 1/len(info["table"][()])}

    return info["table"][par_vals]


### Model Evaluation
I compared predicted classes with actual test labels to measure accuracy and other metrics.  
Metrics like precision and recall show how well the BN distinguishes between benign and malignant cases.  
This gives a clear understanding of the BN’s performance.


In [46]:
y_true = []
y_pred = []

vars = list(train_df.columns)
edges = hill_climb(train_df, vars)  
cpts = learn_cpts(train_df, edges, vars)

for _, row in test_df.iterrows():
    probs = predict_row(cpts, row, target="diagnosis")
    y_true.append(row["diagnosis"])
    pred = max(probs, key=probs.get)
    y_pred.append(pred)

from sklearn.metrics import accuracy_score, classification_report
print("Accuracy:", accuracy_score(y_true, y_pred))
print(classification_report(y_true, y_pred))


Accuracy: 0.8859649122807017
              precision    recall  f1-score   support

      benign       0.88      0.94      0.91        72
   malignant       0.89      0.79      0.84        42

    accuracy                           0.89       114
   macro avg       0.89      0.87      0.87       114
weighted avg       0.89      0.89      0.88       114

